<a href="https://colab.research.google.com/github/nanaoto95s/ML/blob/master/DCGAN_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout
from keras.preprocessing.image import img_to_array, load_img
from keras.optimizers import Adam
import math
import numpy as np
import os
from PIL import Image
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, units=1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(32 * 32 * 128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((32, 32, 128), input_shape=(32 * 32 * 128,)))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (5, 5), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, (5, 5), padding="same"))
    model.add(Activation('tanh'))
    return model
def discriminator_model():
    model = Sequential()
    model.add(Conv2D(64, (5,5), strides=(2, 2), input_shape=(128, 128, 3), padding="same"))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, (5,5), strides=(2, 2)))
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model
def combine_images(generated_images):
    total = generated_images.shape[0]
    cols = int(math.sqrt(total))
    rows = math.ceil(float(total)/cols)
    width, height, ch= generated_images.shape[1:]
    output_shape = (
        height * rows,
        width * cols,
        ch
    )
    combined_image = np.zeros(output_shape)

    for index, image in enumerate(generated_images):
        i = int(index/cols)
        j = index % cols
        combined_image[width*i:width*(i+1), height*j:height*(j+1)] = image[:, :, :]
    return combined_image


BATCH_SIZE = 32
NUM_EPOCH = 1200
GENERATED_IMAGE_PATH = './gdrive/My Drive/generated_images4/' # 生成画像の保存先
TRAIN_IMAGE_PATH = './gdrive/My Drive/train_images/'

def train():
    # 訓練データ読み込み
    img_list = os.listdir(TRAIN_IMAGE_PATH)
    X_train = []
    for img in img_list:
        img = img_to_array(load_img(TRAIN_IMAGE_PATH+img, target_size=(128,128,3)))
        # -1から1の範囲に正規化
        img = (img.astype(np.float32) - 127.5)/127.5
        X_train.append(img)
    # 4Dテンソルに変換(データの個数, 128, 128, 3)
    X_train = np.array(X_train)

    # generatorとdiscriminatorを作成
    discriminator = discriminator_model()
    d_opt = Adam(lr=1e-5, beta_1=0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)
    # discriminatorの重みを固定(dcganの中のみ)
    discriminator.trainable = False
    generator = generator_model()

    dcgan = Sequential([generator, discriminator])
    g_opt = Adam(lr=2e-4, beta_1=0.5)
    dcgan.compile(loss='binary_crossentropy', optimizer=g_opt)

    num_batches = int(X_train.shape[0] / BATCH_SIZE)
    print('Number of batches:', num_batches)
    for epoch in range(NUM_EPOCH):

        for index in range(num_batches):
            noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = generator.predict(noise, verbose=0, batch_size=BATCH_SIZE)

            # 生成画像を出力
            if (index+1) % (num_batches) == 0:
                image = combine_images(generated_images)
                image = image*127.5 + 127.5
                if not os.path.exists(GENERATED_IMAGE_PATH):
                    os.mkdir(GENERATED_IMAGE_PATH)
                Image.fromarray(image.astype(np.uint8))\
                    .save(GENERATED_IMAGE_PATH+"%04d_%04d.png" % (epoch, index))

            # discriminatorを更新
            X = np.concatenate((image_batch, generated_images))
            # 訓練データのラベルが1、生成画像のラベルが0になるよう学習する
            y = [1]*BATCH_SIZE + [0]*BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)

            # generator更新
            noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            # 生成画像をdiscriminatorにいれたときに
            # 出力が1に近くなる(訓練画像と識別される確率が高くなる)ように学習する
            g_loss = dcgan.train_on_batch(noise, [1]*BATCH_SIZE)

            print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" % (epoch, index, g_loss, d_loss))

        generator.save_weights('generator.h5')
        discriminator.save_weights('discriminator.h5')

In [0]:
train()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








Number of batches: 17


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'




epoch: 0, batch: 0, g_loss: 0.529374, d_loss: 0.721783


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, batch: 1, g_loss: 0.640991, d_loss: 0.635636
epoch: 0, batch: 2, g_loss: 0.688177, d_loss: 0.605072
epoch: 0, batch: 3, g_loss: 0.638295, d_loss: 0.629241
epoch: 0, batch: 4, g_loss: 0.558168, d_loss: 0.680402
epoch: 0, batch: 5, g_loss: 0.564658, d_loss: 0.723854
epoch: 0, batch: 6, g_loss: 0.542046, d_loss: 0.660682
epoch: 0, batch: 7, g_loss: 0.524637, d_loss: 0.564728
epoch: 0, batch: 8, g_loss: 0.500010, d_loss: 0.574108
epoch: 0, batch: 9, g_loss: 0.417706, d_loss: 0.602433
epoch: 0, batch: 10, g_loss: 0.466307, d_loss: 0.675606
epoch: 0, batch: 11, g_loss: 0.484073, d_loss: 0.664933
epoch: 0, batch: 12, g_loss: 0.502712, d_loss: 0.696557
epoch: 0, batch: 13, g_loss: 0.538681, d_loss: 0.649062
epoch: 0, batch: 14, g_loss: 0.544749, d_loss: 0.623996
epoch: 0, batch: 15, g_loss: 0.527702, d_loss: 0.707563
epoch: 0, batch: 16, g_loss: 0.527952, d_loss: 0.619522
epoch: 1, batch: 0, g_loss: 0.511814, d_loss: 0.577078
epoch: 1, batch: 1, g_loss: 0.576769, d_loss: 0.536081
epo